# Our Naive Bayes Classifier in 4 stages


##### Stage 1: Preparing the data
    - Key characteristic: powerful normalization step

##### Stage 2: Training and testing
    - Key characteristic: speed

##### Stage 3: Feature selection
    - Key characteristic: easy top n words per class 

##### Stage 4: Evaluation
    - Key characteristic: 

## Stage 1: Preparing data
    - Key characteristic: powerful normalization step

Voormalige top 10
<pre>
Justitie (JUS)
Volksgezondheid, Welzijn en Sport (VWS)
Buitenlandse Zaken (BUZA)
Verkeer en Waterstaat (VW)
Justitie
Sociale Zaken en Werkgelegenheid (SZW)
Onderwijs, Cultuur en Wetenschappen (OCW)
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)
FinanciÃ«n (FIN)
Volksgezondheid, Welzijn en Sport
</pre>

Nieuwe top 10, 33% meer samples
<pre>
justitie
volksgezondheid, welzijn en sport
buitenlandse zaken
verkeer en waterstaat
sociale zaken en werkgelegenheid
onderwijs, cultuur en wetenschappen
volkshuisvesting, ruimtelijke ordening en milieubeheer
financi\xebn
economische zaken
landbouw, natuurbeheer en visserij
</pre>

## Stage 2: Training and testing
    - Key characteristic: speed

In [4]:
from naive_bayes import *
import time

start = time.time()
# Open csv - SAMPLE CODE
kvrdf= open_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz')
read_pandas = time.time()
print 'Read csv: %d' % (read_pandas - start)

Read csv: 11


In [2]:
# Prepare corpus
start = time.time()
documents, ministeries, classes = get_corpus(kvrdf, 10)
train_docs, train_mins, test_docs, test_mins = split_data(documents, ministeries)
corpus_done = time.time()
print 'Prepare corpus: %d' % (corpus_done - start)

# Train NB classifiers for 10 classes 
vocab, prior, cond_prob = train_multinomial_NB(classes, train_docs, train_mins)
trained = time.time()
print 'Train classifier: %d' % (trained - corpus_done)

# Test NB classifier
pred_classes = test_NB(classes, vocab, prior, cond_prob, test_docs, test_mins)
tested = time.time()
print 'Test classifier: %d' % (tested - trained)

print(f1(distribution(classes, test_mins, pred_classes),classes))

Prepare corpus: 2
Train classifier: 4
Test classifier: 5
0.697188309286


## Stage 3: Feature Selection
    - Key characteristic: easy top n words per class 
    

In [ ]:
MI = get_mutual_infs(train_docs, train_mins)

In [5]:
def top_mutual_infs(mutual_infs,n):
    """
    Given the mutual information and interger n, 
    return a dict of top n words based on the mutual information.
    """
    ministeries = {}
    for mini in mutual_infs:
        lst = [(mutual_infs[mini][term],term) 
                   for term in mutual_infs[mini]]
        ministeries[mini] = sorted(lst, reverse=True)[:n]
        
    return ministeries

In [ ]:
def feature_selection(vocabulary,MI, nr_words):
    """
    Using Mutual Information select the top nr_words 
    for classification and return a new vocabulary using those words. 
    """
    top = top_mutual_infs(MI, nr_words)
    return set(word[1] for c in top for word in top[c])

<center><img src='F1 plot.png' /></center>

## Stage 4: Evaluation
    - Key characteristic: Single computation of:
        - True Positive
        - False Positive
        - False Negative

In [ ]:
def distribution(classes, test_mins, pred_classes):
    """
    Calculate the True Positives, False Positives, and False Negatives per class 
    and return a dict with a counter as value for each class. 
    """
    data = { c : Counter() for c in classes }
    
    for c in classes:
        for i, ministerie in enumerate(test_mins):
            if (c == ministerie and c == pred_classes[i]):
                data[c]['TP'] += 1
            elif (c == pred_classes[i] and c != ministerie):
                data[c]['FP'] += 1
            elif (c != pred_classes[i] and c == ministerie):
                data[c]['FN'] += 1
            
    return data

# Questions?